In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

In [3]:
#importing the new york dataset for analysis
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


<h4>Transforming data into pandas dataframe:</h4>

In [4]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    #by default borough names will be taken as neighborhood names 
    borough = neighborhood_name = data['properties']['borough'] 
    #if neighborhood names of the boroughs are defined in the dataset, they will be stored in neighborhood name column 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [5]:
#printing first 5 rows of the dataframe
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


<h4>Use geopy library to get the latitude and longitude values of New York City.<br>
In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.</h4>

In [6]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of New York City are 40.7127281, -74.0060152.


<h4>Create a map of New York with neighborhoods superimposed on top, focusing on borough name 'Manhattan', where the restaurant needs to open.</h4>

In [7]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
#display first 5 rows
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [8]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

<h4>Foursquare venues:</h4>

In [9]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                          v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [10]:
#connecting to Foursquare api to obtain Korean restaurants in Manhattan within a radius of 1000m
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'DT3LGTV1USPKRTGNS4MW3RXDLMJFRDWRTLLWSU1F4EIT2QXW'
CLIENT_SECRET = '3JHOQ454USOUNS25GD0QQ5TXX0TNIVAPKY0ENPONHOJOXHGR'
VERSION = '20200516'

#https://developer.foursquare.com/docs/resources/categories
#Korean Restaurant = 4bf58dd8d48988d113941735
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_korean = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d113941735')

#printing first 5 rows
newyork_venues_korean.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,KBBQ,40.868843,-73.901956,Korean Restaurant
1,Chinatown,40.715618,-73.994279,Grand Seoul,40.717447,-73.994256,Korean Restaurant
2,Chinatown,40.715618,-73.994279,88 Kimbap,40.718192,-73.995926,Korean Restaurant
3,Chinatown,40.715618,-73.994279,Nowon,40.725390,-73.983475,Korean Restaurant
4,Chinatown,40.715618,-73.994279,To Two Boonsik,40.715517,-73.993932,Korean Restaurant


In [11]:
#Number of Korean restaurants in Manhattan within radius of 1000m  (It shows 619 restaurants as of date)
newyork_venues_korean.shape

(619, 7)

In [12]:
#adding Korean restaurants to the existing map of New York
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [73]:
#generating the map with Korean restaurants
map_newyork_korean = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_korean, 'red', map_newyork_korean)

map_newyork_korean

In [13]:
#Grouping the Korean Restaurant venues by neighborhoods for easy analysis, to checkout which areas have max and min no. of Korean joints
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [19]:
manhattan_grouped = newyork_venues_korean.groupby('Neighborhood').count()

#print the venue count of Korean restaurants in each neighborhood of Manhattan, NY.
manhattan_grouped

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,7,7,7,7,7,7
Carnegie Hill,2,2,2,2,2,2
Chelsea,23,23,23,23,23,23
Chinatown,12,12,12,12,12,12
Civic Center,15,15,15,15,15,15
Clinton,26,26,26,26,26,26
East Village,24,24,24,24,24,24
Financial District,6,6,6,6,6,6
Flatiron,50,50,50,50,50,50


<h1>Analyzing each neighborhood</h1>

In [36]:
#Sorting neighborhood based on count of Korean Restaurants
manhattan_grouped.sort_values(by ='Venue')

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Roosevelt Island,1,1,1,1,1,1
Marble Hill,1,1,1,1,1,1
Yorkville,2,2,2,2,2,2
Carnegie Hill,2,2,2,2,2,2
Upper East Side,2,2,2,2,2,2
Manhattanville,2,2,2,2,2,2
Lenox Hill,4,4,4,4,4,4
Manhattan Valley,4,4,4,4,4,4
Morningside Heights,6,6,6,6,6,6


<h4>We can see that first 6 neighborhoods (Roosevelt Island, Marble Hill, Yorkville, Carnegie Hill, Upper East Side, Manhattanville) have the least no. of Korean Restaurants.</h4>

In [38]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_korean[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_korean['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Asian Restaurant,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Grocery Store,Japanese Restaurant,Korean Restaurant,Lounge,Miscellaneous Shop,Poke Place,Salad Place,Vegetarian / Vegan Restaurant,Wings Joint
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [39]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Asian Restaurant,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Grocery Store,Japanese Restaurant,Korean Restaurant,Lounge,Miscellaneous Shop,Poke Place,Salad Place,Vegetarian / Vegan Restaurant,Wings Joint
0,Battery Park City,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.00,0.000000,0.142857,0.000000,0.000000,0.000000,0.571429,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000
1,Carnegie Hill,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Chelsea,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.00,0.043478,0.000000,0.000000,0.000000,0.000000,0.913043,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Chinatown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Civic Center,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.066667,0.000000,0.000000,0.000000,0.800000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000
5,Clinton,0.000000,0.038462,0.000000,0.038462,0.000000,0.000000,0.00,0.038462,0.038462,0.076923,0.000000,0.000000,0.730769,0.000000,0.000000,0.000000,0.038462,0.000000,0.000000
6,East Village,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.958333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Financial District,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.00,0.000000,0.333333,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000
8,Flatiron,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.000000,0.000000,0.020000,0.000000,0.000000,0.940000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Gramercy,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.021277,0.000000,0.000000,0.000000,0.000000,0.893617,0.021277,0.021277,0.000000,0.000000,0.021277,0.000000


In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Korean Restaurant,Poke Place,Deli / Bodega,Food Truck,Wings Joint,Food Stand,BBQ Joint,Cocktail Bar,Coffee Shop,Food & Drink Shop
1,Carnegie Hill,Korean Restaurant,Wings Joint,Food Truck,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop,Food Court,Food Stand
2,Chelsea,Korean Restaurant,Cocktail Bar,Food Stand,Wings Joint,Food Truck,BBQ Joint,Coffee Shop,Deli / Bodega,Food & Drink Shop,Food Court
3,Chinatown,Korean Restaurant,Wings Joint,Food Truck,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop,Food Court,Food Stand
4,Civic Center,Korean Restaurant,Asian Restaurant,Poke Place,Food Truck,Food Stand,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop


<h4>Cluster the neighborhoods: </h4>

In [74]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 1, 1, 2, 4, 1, 0, 1, 2], dtype=int32)

In [81]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
manhattan_merged = manhattan_merged.dropna()
manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1.0,Korean Restaurant,Wings Joint,Food Truck,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop,Food Court,Food Stand
1,Manhattan,Chinatown,40.715618,-73.994279,1.0,Korean Restaurant,Wings Joint,Food Truck,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop,Food Court,Food Stand
5,Manhattan,Manhattanville,40.816934,-73.957385,1.0,Korean Restaurant,Wings Joint,Food Truck,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop,Food Court,Food Stand
8,Manhattan,Upper East Side,40.775639,-73.960508,1.0,Korean Restaurant,Wings Joint,Food Truck,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop,Food Court,Food Stand
9,Manhattan,Yorkville,40.775930,-73.947118,1.0,Korean Restaurant,Wings Joint,Food Truck,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop,Food Court,Food Stand


In [87]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [60]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Battery Park City,Korean Restaurant,Poke Place,Deli / Bodega,Food Truck,Wings Joint,Food Stand,BBQ Joint,Cocktail Bar,Coffee Shop,Food & Drink Shop
29,Financial District,Korean Restaurant,Food Truck,Poke Place,Deli / Bodega,Wings Joint,Food Stand,BBQ Joint,Cocktail Bar,Coffee Shop,Food & Drink Shop


In [61]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Korean Restaurant,Wings Joint,Food Truck,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop,Food Court,Food Stand
1,Chinatown,Korean Restaurant,Wings Joint,Food Truck,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop,Food Court,Food Stand
5,Manhattanville,Korean Restaurant,Wings Joint,Food Truck,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop,Food Court,Food Stand
8,Upper East Side,Korean Restaurant,Wings Joint,Food Truck,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop,Food Court,Food Stand
9,Yorkville,Korean Restaurant,Wings Joint,Food Truck,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop,Food Court,Food Stand
10,Lenox Hill,Korean Restaurant,Wings Joint,Food Truck,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop,Food Court,Food Stand
11,Roosevelt Island,Korean Restaurant,Wings Joint,Food Truck,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop,Food Court,Food Stand
17,Chelsea,Korean Restaurant,Cocktail Bar,Food Stand,Wings Joint,Food Truck,BBQ Joint,Coffee Shop,Deli / Bodega,Food & Drink Shop,Food Court
19,East Village,Korean Restaurant,Asian Restaurant,Food Truck,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop,Food Court,Food Stand
20,Lower East Side,Korean Restaurant,Wings Joint,Food Truck,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop,Food Court,Food Stand


In [62]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Midtown,Korean Restaurant,Food Truck,Fried Chicken Joint,Food Court,Salad Place,BBQ Joint,Lounge,Coffee Shop,Deli / Bodega,Vegetarian / Vegan Restaurant
16,Murray Hill,Korean Restaurant,Food Truck,Fried Chicken Joint,Food Court,Salad Place,Lounge,Vegetarian / Vegan Restaurant,BBQ Joint,Cocktail Bar,Coffee Shop
18,Greenwich Village,Korean Restaurant,Asian Restaurant,Japanese Restaurant,Food Truck,Food Stand,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop
21,Tribeca,Korean Restaurant,Asian Restaurant,Japanese Restaurant,Food Truck,Food Stand,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop
22,Little Italy,Korean Restaurant,Asian Restaurant,Food Truck,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop,Food Court,Food Stand
23,Soho,Korean Restaurant,Asian Restaurant,Japanese Restaurant,Food Truck,Food Stand,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop
24,West Village,Korean Restaurant,Asian Restaurant,Japanese Restaurant,Food Truck,Food Stand,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop
27,Gramercy,Korean Restaurant,Asian Restaurant,Miscellaneous Shop,Lounge,Vegetarian / Vegan Restaurant,Food Stand,Food Court,BBQ Joint,Cocktail Bar,Coffee Shop
32,Civic Center,Korean Restaurant,Asian Restaurant,Poke Place,Food Truck,Food Stand,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop
33,Midtown South,Korean Restaurant,Fried Chicken Joint,Food Court,Salad Place,Lounge,Vegetarian / Vegan Restaurant,Food Truck,BBQ Joint,Cocktail Bar,Coffee Shop


In [63]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Manhattan Valley,Korean Restaurant,Wings Joint,Grocery Store,Food Stand,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop,Food Court
26,Morningside Heights,Korean Restaurant,Wings Joint,Grocery Store,Food Stand,BBQ Joint,Cocktail Bar,Coffee Shop,Deli / Bodega,Food & Drink Shop,Food Court


In [64]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Lincoln Square,Korean Restaurant,Fried Chicken Joint,Coffee Shop,Food Truck,BBQ Joint,Cocktail Bar,Deli / Bodega,Food & Drink Shop,Food Court,Food Stand
14,Clinton,Korean Restaurant,Fried Chicken Joint,Food Stand,Salad Place,BBQ Joint,Coffee Shop,Food Truck,Cocktail Bar,Deli / Bodega,Food & Drink Shop
39,Hudson Yards,Korean Restaurant,Fried Chicken Joint,Food Stand,Salad Place,BBQ Joint,Coffee Shop,Cocktail Bar,Deli / Bodega,Food & Drink Shop,Food Court
